# MeltPool Surrogate Simulator

In [ ]:
# importing necessary libraries and 
import numpy as np
np.seterr(all='ignore')
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device is', Device)

from Dataset import AM_3D_Dataset
from utils import get_samples, get_train_val, viz_dataset, get_meshes, what_is
from GenClass import GenClass

Tevap = 2500
room = 293


In [ ]:
data = 'Ti64-5'  # 'Ti64-5' or 'Ti64-10' or 'Ti64-10-p'
masked = True

samples = get_samples(data)
train_idxs, val_idxs = get_train_val(samples, data)
%matplotlib inline
viz_dataset(samples, val=val_idxs, s=25, annotate=10)

## Visualizing data samples

In [ ]:
from utils import load, get_pv, remake, viz_sample, viz_process

In [ ]:
# Choose sample index and time step (from 1 to 99)
sample_idx = 40
t = 70

#run this cell to load the data
sample = samples[sample_idx]
p, v = get_pv(sample)
T = remake(*load(sample, data, t), data)

print(f'P = {p} w , V = {v} mm/s , t = {t}')

In [ ]:
%matplotlib inline
viz_sample(T, data, figsize=(6.5,8))

In [ ]:
# Choose a sample index for an animated visualization
sample_idx = 40

# run this cell to load the data for the whole process (all time steps)
p, v = get_pv(sample)
Ts = []
for t in range(1, 100):
    T = remake(*load(sample, data, t), data)
    Ts.append(T)
Ts = np.stack(Ts)

In [ ]:
print(f'P = {p} w , V = {v} mm/s')
%matplotlib notebook
# If the animation does not show up, run the cell a couple of times!
viz_process(Ts, data, figsize=(7,9))

## Loading a trained model

In [ ]:
models = [model for model in os.listdir('Results/'+data) if model.startswith('FC')]
print('Here is the list of the trained models for this data:\n')
for i, model in enumerate(models):
    Model = GenClass(model, data)
    print(f'Model {i:02d}:  {model}')

In [ ]:
# Choose the model index
model_idx = 0

# Load the trained model
Model = GenClass(models[model_idx], data)
Model.load_state_dict(masked=masked)
Model.test_mode()

# Load the dataset (this might take a while)
Model.set_dataset(
    AM_3D_Dataset(samples[train_idxs], data), # training data
    AM_3D_Dataset(samples[val_idxs], data) # validation data
)

## Trying the model on samples from the dataset

In [ ]:
from utils import compare_samples, compare_process, compare_sample

In [ ]:
# you can choose the index from the dataset plot in the beginning o the notebook
# run this cell to obtain model output and ground truth data

sample_idx = 40

x, Ts, Ts_pred = Model.val_process(samples[sample_idx], masked=masked)
p, v = x
print(f'P = {p} w , V = {v} mm/s')

In [ ]:
# Compare Model output with ground truth for certain timesteps

save = '' # Model.outputs_dir+'/'+f'{x[:2]}.jpg'
%matplotlib inline
fig = compare_samples(x, Ts, Ts_pred, data, save=save, ts=[10, 30, 50, 70, 90])

In [ ]:
# Compare an animation of the process (the middle cross-section) with error plot

save = '' # Model.outputs_dir+'/'+f'{x[:2]}.mp4'
%matplotlib notebook
# If the animation does not show up, run the cell a couple of times!
ani = compare_process(x, Ts, Ts_pred, data, save=save)

In [ ]:
# Compare a specific time step of a sample with error plot
# set the time step to show:
t = 30
save = '' # Model.outputs_dir+'/'+f'{x[:2]}_t({t}).jpg'
%matplotlib inline
fig = compare_sample((p, v, t), Ts, Ts_pred, data, save=save)

## Get the results for the whole dataset

In [ ]:
from utils import get_rmse_iou, plot_result_vs_t, plot_result_vs_PV
RMSE, IoU = [], []
for idx in tqdm(range(len(samples)), position=0, leave=True):
    # We load the data one by one due to memory limitations
    Model.set_dataset(None, AM_3D_Dataset(samples[idx:idx+1], data))
    (p, v), Ts, Ts_pred = Model.val_process(samples[idx], masked=masked)
    rmse, iou = get_rmse_iou(Ts, Ts_pred)
    RMSE.append(rmse)
    IoU.append(iou)
RMSE = np.stack(RMSE)
IoU = np.stack(IoU)

In [ ]:
save = '' # 'Results/' + data + '/results_vs_t.jpg'
%matplotlib inline
plot_result_vs_t(RMSE, IoU, train_idxs, val_idxs, save=save)

In [ ]:
save_name = '' # 'Results/' + data + '/results_vs_PV.jpg'
s, edge = (40, 1) if data == 'Ti64-10-p' else (70, 1.6)
%matplotlib inline
plot_result_vs_PV(RMSE, IoU, samples, train_idxs, val_idxs, s=s, save=save_name, edge=edge, rmse_vmax=10)

In [ ]:
Trange = Tevap-room
with open('Results.txt', 'a+') as f:
    s = 'data: ' + data + (' masked' if masked else '') + '\n'
    s += f'RMSE: {np.nanmean(RMSE):.1f} +- {np.nanstd(RMSE):.1f} K '
    s += f'({100*np.nanmean(RMSE)/Trange:.2f} +- {100*np.nanstd(RMSE)/Trange:.2f} %)' + '\n'
    s += f'IoU: {100*np.nanmean(IoU):.2f} +- ({100*np.nanstd(IoU):.2f} %)' + 2*'\n'
    f.write(s)